<a href="https://colab.research.google.com/github/vishalkesti382/AI_in_Finance/blob/master/Copy_of_Predicting_Movie_Reviews_with_BERT_on_TF_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Fund Name Class with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.



In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [37]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [14]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.4MB/s 


In [15]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [16]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'OUTPUT_DIR_NAME'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'FUND_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: OUTPUT_DIR_NAME *****


#Data

First, let's load our fund name data. 

In [0]:
from tensorflow import keras
import os
import re
import pandas as pd
import numpy as np


# Load all files from a directory in a DataFrame.
# Load and process the dataset files.
def dataset_load():
  dataset = pd.read_csv('/content/Clean_classificaion_data.csv')
  dataset.dropna(inplace =True)
  seed = 929
  labels = dataset['doctype'].values
  train_df, test_df = train_test_split(dataset, test_size=0.3, stratify=labels, shuffle=True, random_state=seed) 
  print('data loaded')
  
  return train_df, test_df


In [0]:
dataset = pd.read_csv('/content/Clean_classificaion_data.csv')

In [10]:
dataset.head()

,Unnamed: 0,doctype,fullfilename,text,preprocessed_text,char_num,word_num
0,0,Red Docs,W:CGSS\InterDept\Digitization\Model Training\N...,\nREDEMPTION REQUEST FORM \n \nTo: Triton Val...,redemption REQUEST FORM To ...,2944,473
1,1,Red Docs,W:CGSS\InterDept\Digitization\Model Training\N...,SC-88707\nCertificate of Registration of Exemp...,SC Certificate of Registration of Exempted Lim...,691,110
2,2,Red Docs,W:CGSS\InterDept\Digitization\Model Training\N...,Citco Fund Services (Singapore) Pte Ltd\nTrans...,Citco Fund Services Singapore Pte Ltd Transfer...,3833,610
3,3,Red Docs,W:CGSS\InterDept\Digitization\Model Training\N...,WC-102342\nCertificate of Registration of Exem...,WC Certificate of Registration of Exempted Lim...,684,110
4,4,Red Docs,W:CGSS\InterDept\Digitization\Model Training\N...,Portcullis Fund Administration \nChange / Retr...,Portcullis Fund Administration Change Retrieve...,1154,183


In [26]:
dataset["doctype"].value_counts()

Trade Contract Note       2386
Statements                2335
Performance Reports       1543
Transfer Contract Note    1009
Trade ACK                  841
Prospectus                 507
Sub Docs                   430
Capital Call Notices       341
Financial Statements       276
Red Docs                   231
Meetings AGM EGM           216
KIID Documents             199
Distribution Notice         66
Dividend Confirmation       66
Corporate Action CN         51
NAV Report                  49
FINRA                        9
Equalisation CN              5
Stock Transfer Forms         5
Compulsory Red CN            5
Liquidations                 3
Switch Contract Note         2
Name: doctype, dtype: int64

In [17]:
train, test = dataset_load()

data loaded


In [0]:
cleanup_nums = {"doctype": {"Trade Contract Note": 0, "Statements": 1, "Performance Reports": 2, "Transfer Contract Note": 3,
                                  "Trade ACK": 4, "Prospectus": 5, "Sub Docs":6, "Capital Call Notices":7, "Financial Statements":8, 
                            "Red Docs":9, "Meetings AGM EGM":10,"KIID Documents":11, "Distribution Notice":12, "Dividend Confirmation":13, "Corporate Action CN":14,
                            "NAV Report": 15, "FINRA":16, "Equalisation CN":17, "Stock Transfer Forms":18, "Compulsory Red CN":19, "Liquidations":20,
                            "Switch Contract Note":21}}

In [28]:
dataset.replace(cleanup_nums, inplace=True)
dataset.head()

,Unnamed: 0,doctype,fullfilename,text,preprocessed_text,char_num,word_num
0,0,9,W:CGSS\InterDept\Digitization\Model Training\N...,\nREDEMPTION REQUEST FORM \n \nTo: Triton Val...,redemption REQUEST FORM To ...,2944,473
1,1,9,W:CGSS\InterDept\Digitization\Model Training\N...,SC-88707\nCertificate of Registration of Exemp...,SC Certificate of Registration of Exempted Lim...,691,110
2,2,9,W:CGSS\InterDept\Digitization\Model Training\N...,Citco Fund Services (Singapore) Pte Ltd\nTrans...,Citco Fund Services Singapore Pte Ltd Transfer...,3833,610
3,3,9,W:CGSS\InterDept\Digitization\Model Training\N...,WC-102342\nCertificate of Registration of Exem...,WC Certificate of Registration of Exempted Lim...,684,110
4,4,9,W:CGSS\InterDept\Digitization\Model Training\N...,Portcullis Fund Administration \nChange / Retr...,Portcullis Fund Administration Change Retrieve...,1154,183


In [0]:
seed = 929
labels = dataset['doctype'].values
train, test = train_test_split(dataset, test_size=0.3, stratify=labels, shuffle=True, random_state=seed)

In [30]:
train.shape

(7402, 7)

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [0]:
train = train.sample(3000)
test = test.sample(2000)

In [33]:
train.columns

Index(['Unnamed: 0', 'doctype', 'fullfilename', 'text', 'preprocessed_text',
       'char_num', 'word_num'],
      dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [0]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'doctype'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'fundName', 'Not_fundName'
label_list = [0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [36]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [37]:
tokenizer.tokenize("Bridgewater Pure Alpha Fund I Everett, MA 02149 Class CLASS B")

['bridge',
 '##water',
 'pure',
 'alpha',
 'fund',
 'i',
 'everett',
 ',',
 'ma',
 '02',
 '##14',
 '##9',
 'class',
 'class',
 'b']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [38]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 3000


INFO:tensorflow:Writing example 0 of 3000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 11 (id = 11)


INFO:tensorflow:label: 11 (id = 11)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [44]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3fbc905940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3fbc905940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [46]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 281 into OUTPUT_DIR_NAME/model.ckpt.


INFO:tensorflow:Saving checkpoints for 281 into OUTPUT_DIR_NAME/model.ckpt.


INFO:tensorflow:Loss for final step: 0.52769756.


INFO:tensorflow:Loss for final step: 0.52769756.


Training took time  0:08:42.558131


Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [48]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


InvalidArgumentError: ignored

Now let's write code to make predictions on new sentences:

In [0]:
def getPrediction(in_sentences):
  labels = ["Trade Contract Note", "Statements","Performance Reports", "Transfer Contract Note", "Trade ACK", "Prospectus","Sub Docs",                   
"Capital Call Notices", "Financial Statements", "Red Docs", "Meetings AGM EGM", "KIID Documents", "Distribution Notice",         
"Dividend Confirmation", "Corporate Action CN", "NAV Report", "FINRA", "Equalisation CN", "Stock Transfer Forms", "Compulsory Red CN",            
"Liquidations", "Switch Contract Note" ]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = ["CAPITAL CALL NOTIFICATION LETTER Global Infrastructure Solutions 3",
"Feeder SCSP July 20, 2018 Dear Investor In accordance with Sections 3.1 and 4.1 of the Amended and",
"Restated Deed of Limited Partnership, we are writing to inform you of the Global Infrastructure Solutions 3 Feeder SCSp’s capital call.",
"The capital call is primarily being drawn down to fund new and follow-on investments. Your capital call details are located on the accompanying notice.",
"The amount due is payable on August 03, 2018 in Euros. Sincerely yours, Serge Lauper Managing Director BlackRock Infrastructure INV ID# 21138394", 
"Global Infrastructure Solutions 3 Feeder SCSp Capital Call Notification ",
"July 20, 2018 PAYMENT DUE AUGUST 03, 2018 Clearstream Banking S.A. AFS 145326 This mailing constitutes a capital call notice for Global ",
"Infrastructure Solutions 3 Feeder SCSp. Please note the total amount due and ensure that the capital call is fully funded no later than the due date. ",
"Capital Call Components - Cash 1 Investments 174,548 Total Capital Call € 174,548 Partner's ",
"Capital Activity Summary Commitment 100.00 % 3,000,000 Funded commitment before this capital call 33.96 % 1,018,765 Current capital call", 
"5.82 % 174,548 Funded commitment after this capital call 39.78 % 1,193,313 Unfunded commitment after this capital call 60.22 %   € 1,806,687 ",
"Please use the instructions below for paying your capital call. When remitting your payment, please ensure that adequate provision is made for",
 "bank charges and transfer costs. Name of Bank: Deutsche Bank AG Swift Number: DEUTDEFFXXX SBOSGB2XXXX, State Street Bank & Trust Account Name: ",
 "Company, London Treasury IBAN: DE43 5007 0010 0927 361600 Amount Due: € 174,548 Due Date: August 03, 2018 BLTNEUR01 - ",
 "Clearstream Banking S.A. AFS Reference: 145326 To properly allocate your payment to our bank account please include BLTNEUR01 in the reference ",
" of the payment. Should you have any questions, please do not hesitate to contact us at +44-207-743-2603 or via email    realassets@blackrock.com. ",
 "Also, to ensure the accuracy of the Fund’s records and your timely receipt of all capital call notification, cash distributions and general",
 " correspondence, please report all changes promptly. 1 Decreases unfunded commitment INV ID# 21138394    Copyright © 2018 BlackRock. All Rights Reserved. "]




In [53]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 18


INFO:tensorflow:Writing example 0 of 18


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from OUTPUT_DIR_NAME/model.ckpt-281


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Great! We have a Fund name classifier!

In [54]:
predictions

[('CAPITAL CALL NOTIFICATION LETTER Global Infrastructure Solutions 3',
  array([-4.829085  , -4.5385303 , -3.588954  , -4.952178  , -4.056877  ,
         -5.4448214 , -4.3627553 , -0.42157695, -5.0326505 , -2.5146303 ,
         -3.650778  , -5.9266686 , -3.8745835 , -4.3029766 , -4.1463785 ,
         -4.142169  , -4.015101  , -4.9833565 , -4.49714   , -3.9920034 ,
         -4.8230915 , -4.452085  ], dtype=float32),
  'Capital Call Notices'),
 ('Feeder SCSP July 20, 2018 Dear Investor In accordance with Sections 3.1 and 4.1 of the Amended and',
  array([-4.938771 , -4.2299557, -2.0524607, -4.472643 , -4.798257 ,
         -1.5666049, -3.648292 , -1.3724289, -3.8219516, -3.478047 ,
         -1.603549 , -4.167856 , -4.8630943, -4.928856 , -4.7478533,
         -4.4528947, -4.8424487, -5.5739536, -5.0385838, -4.859559 ,
         -5.1754646, -5.175075 ], dtype=float32),
  'Capital Call Notices'),
 ('Restated Deed of Limited Partnership, we are writing to inform you of the Global Infrastructu

In [0]:
!ls {'OUTPUT_DIR_NAME'}

checkpoint				     model.ckpt-0.index
eval					     model.ckpt-0.meta
events.out.tfevents.1576259185.f74618c4161e  model.ckpt-468.data-00000-of-00001
graph.pbtxt				     model.ckpt-468.index
model.ckpt-0.data-00000-of-00001	     model.ckpt-468.meta


In [0]:
latest = tf.train.latest_checkpoint('OUTPUT_DIR_NAME')


In [0]:
latest

'OUTPUT_DIR_NAME/model.ckpt-468'

In [0]:
# Create a new model instance
model = create_model(predicting, input_ids, input_mask, segment_ids, labels,num_labels)

# Load the previously saved weights
# model.load_weights(latest)

NameError: ignored

In [0]:
# Save the weights
estimator.save_weights('OUTPUT_DIR_NAME')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('OUTPUT_DIR_NAME')


NameError: ignored

In [0]:
!mkdir -p saved_model

In [0]:
!pip install -q pyyaml h5py